In [47]:
%load_ext autoreload
%autoreload 2

from NLP_Lib import *

import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import math
import numpy as np
from numpy import dtype

import tensorflow_text as text
import tensorflow_hub as hub


from tensorflow.data import Dataset

# disable eager execution to use ELMo model
# tf.disable_eager_execution()

# improve/change plot appearance
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# load in the training data, make sure it is the form we expect
training_df = pd.read_table(PATH_TO_TRAINING_TSV)
assert (training_df.shape[1] > 1 and CLASS_COL == training_df.columns[1]
    and training_df.shape[1] > 2 and SUBCLASS_COL == training_df.columns[2]
    and training_df.shape[1] > 3 and TEXT_COL == training_df.columns[3])

# shuffle
training_df = training_df.sample(frac=1).reset_index(drop=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    PATH_TO_TRAINING_DATASET_STRUCT,
    batch_size=batch_size,
    label_mode="categorical",
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    PATH_TO_TRAINING_DATASET_STRUCT,
    batch_size=batch_size,
    label_mode="categorical",
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = build_classifier_model(SMALL_BERT[0], SMALL_BERT[1])
model.compile(
    optimizer=tf.keras.optimizers.Adam(), # SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax Nadam, Ftrl
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"])

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

checkpoint_dir_x = os.path.join(CHECKPOINT_DIR, datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
pathlib.Path(checkpoint_dir_x).mkdir(parents=True, exist_ok=True)
callbacks = [
    keras.callbacks.ModelCheckpoint(os.path.join(checkpoint_dir_x, "save_at_{epoch}.h5")),
]

model.summary()

In [ ]:
epochs = 5
history = model.fit(
    x=train_ds.with_options(options),
    validation_data=val_ds.with_options(options),
    verbose=2, # 0 is silent, 1 for loading bar, 2 for stats each epoch
    epochs=epochs)


plot_accuracy(plt, history, checkpoint_dir_x)
plot_loss(plt, history, checkpoint_dir_x)